# MindSpore入门，鸢尾花分类
本实验将使用MindSpore深度学习框架，使用鸢尾花数据集，搭建简单的全连接神经网络，完成鸢尾花种类分类任务。

鸢尾属约有300个品种，本实验将对下列3个品种进行分类：
* setosa
* versicolor
* virginica

数据集包含4个特征：sepal_length、sepal_width、petal_length、petal_width

标签中0代表setosa、1代表versicolor、2代表virginica

<img src="image/01.png">

MindSpore文档：https://www.mindspore.cn/doc/api_python/zh-CN/r1.1/index.html

本实验需要以下第三方库：
1. MindSpore 1.7
2. Numpy 1.17.5
3. Scikit-learn 0.24.1

环境导入

In [5]:
import mindspore.dataset as ds # 载入数据集
import mindspore.nn as nn # 网络相关
from mindspore.nn.loss import SoftmaxCrossEntropyWithLogits # 损失函数
from mindspore.nn.metrics import Accuracy # 评估矩阵
from mindspore import Model

import os # 文件路径处理
import numpy as np # numpy
from sklearn.model_selection import train_test_split # 数据集划分

首先，我们用numpy读取csv格式的数据。

In [6]:
from numpy import genfromtxt
iris_data = genfromtxt('iris.csv', delimiter=',')
print(iris_data[:10]) # 查看前10笔数据

[[nan nan nan nan nan]
 [5.1 3.5 1.4 0.2 0. ]
 [4.9 3.  1.4 0.2 0. ]
 [4.7 3.2 1.3 0.2 0. ]
 [4.6 3.1 1.5 0.2 0. ]
 [5.  3.6 1.4 0.2 0. ]
 [5.4 3.9 1.7 0.4 0. ]
 [4.6 3.4 1.4 0.3 0. ]
 [5.  3.4 1.5 0.2 0. ]
 [4.9 3.1 1.5 0.2 0. ]]


iris_data中的第1行不需要，前4列是特征，最后1列是标签。

我们需要对iris_data做简单处理，并划分数据集。

In [7]:
iris_data = iris_data[1:] # 移除第一行
X = iris_data[:,:4].astype(np.float32) # 特征
y = iris_data[:,-1].astype(np.int32) # 标签

# 数据归一化
X /= np.max(np.abs(X),axis=0)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

最后，我们将数据转换成MindSpore的Dataset格式。

In [8]:
# 训练集
train_data = (X_train, y_train)
train_data = ds.NumpySlicesDataset(train_data)

# 测试集
test_data = (X_test, y_test)
test_data = ds.NumpySlicesDataset(test_data)

# 批处理
train_data = train_data.batch(32)
test_data = test_data.batch(32)

完成数据处理后，我们开始定义网络。

这里的网络包含输入层、1个隐藏层和输出层，激活函数选择ReLU。

<img src="image/02.png">

In [9]:
class my_net(nn.Cell):
    # 定义算子
    def __init__(self):
        super(my_net, self).__init__()
        self.fc1 = nn.Dense(4, 10) # 全连接层
        self.fc2 = nn.Dense(10, 3) # 全连接层
        self.relu = nn.ReLU() # 激活函数
        
    # 建构网络
    def construct(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

定义好网络后，我们接着建立模型。

除了网络，我们的模型还需要损失函数和优化器。

In [10]:
# 网络
net = my_net()

# 损失函数
net_loss = SoftmaxCrossEntropyWithLogits(sparse=True) # sparse，输出不是one hot编码时设为Ture

# 优化器
lr = 0.01 # 学习率
momentum = 0.9 # 动量
net_opt = nn.Momentum(net.trainable_params(), lr, momentum)

# 模型
model = Model(net, net_loss, net_opt, metrics={"accuracy": Accuracy()})

模型建立好后，就能开始训练。

这里设置10次迭代。

❋ 如果看到dataset sink mode的警告可以忽略，dataset sink mode将在后续介绍。

In [11]:
model.train(10, train_data)

最后，我们用测试集评估模型的准确率。

In [12]:
model.eval(test_data)

{'accuracy': 0.9666666666666667}

## 思考
1. 在对iris_data做简单处理时，为何要将最后一列数据改为int形态？
    * 答：因为最后一列数据是标签，本实验所使用的损失函数（SoftmaxCrossEntropyWithLogits(sparse=True)，分类任务常用损失函数）需要int形态的标签。
1. 什么是一个epoch？
    * 答：一个epoch指所有的数据送入网络中完成一次前向计算及反向传播的过程。
2. 本实验使用SoftmaxCrossEntropyWithLogits(sparse=True)损失函数，其中sparse参数为何要设置成Ture？
    * 答：本实验使用的标签是int形态，CrossEntropy损失函数需要one_hot形式的标签，当sparse为True时MindSpore会对标签做one_hot处理。
3. SoftmaxCrossEntropyWithLogits和SoftmaxCrossEntropy分别在什么情况下使用？
    * 答：当输出层没有sofmax激活函数时，使用SoftmaxCrossEntropyWithLogits。若输出层有sofmax激活函数，使用SoftmaxCrossEntropy。
4. 本实验为分类任务，如果进行回归任务，可以设置什么损失函数？
    * 答：smoothl1loss，MSELoss（暂不支持CPU）等。
5. 如果进行回归任务，最后的输出通道数是多少？
    * 答：1